<h1> Custom Environment </h1>
<h4> -An attempt at creating it from scratch </h4>
<i> "You either die a hero, or live long enough to see yourself become a villain."</i>

class
<br>
initializer
<br>
step
<br>
reset
<br>
render
<br>
close
<br>
<br>
Validate thorugh check_env()

https://colab.research.google.com/github/araffin/rl-tutorial-jnrr19/blob/master/5_custom_gym_env.ipynb#scrollTo=9DOpP_B0-LXm

In [52]:
import numpy as np
import pandas as pd

import gymnasium as gym
from gymnasium import spaces

from stable_baselines3.common.env_checker import check_env

from stable_baselines3 import SAC
from stable_baselines3 import PPO
from stable_baselines3 import RecurrentPPO

ImportError: cannot import name 'RecurrentPPO' from 'stable_baselines3' (c:\Users\Jakob\OneDrive\Dokumenter\Masteroppgave\MyVenv\Lib\site-packages\stable_baselines3\__init__.py)

In [2]:
history_usage = 30

return_data = pd.read_csv("../../Data/StockReturns.csv")

esg_data = np.array([36.6, 35.3, 17.9, 18, 
                    18, 21.2, 18.7, 29.2, 
                    15.7, 25.6, 25.6, 18.4, 
                    19.8, 13.8, 18.1, 19, 
                    17.2, 14, 17.2, 19.5, 
                    19.7, 21.2, 26.8, 19.3])

objective = "Sharpe"

esg_compliancy = True

In [3]:
split_size = 0.8

train_data = return_data.iloc[:int(split_size*len(return_data))]
test_data = return_data.iloc[int(split_size*len(return_data)):].reset_index(drop=True)

In [32]:
class PortfolioEnvironment(gym.Env):
    """
    doc string
    """
    def __init__(self,
                 history_usage,
                 return_data, esg_data,
                 objective, esg_compliancy):
        super().__init__()
        """
        doc  string,

        Good, initialize all variables with values 
        """
        self.return_data = return_data.values
        self.esg_data: np.array = esg_data
        self.history_usage: int = history_usage
        self.n_stocks = len(esg_data)

        self.objective: str = objective
        self.esg_compliancy: bool = esg_compliancy

        self.action_space = spaces.Box(low=-1, 
                                       high=1, 
                                       shape=(self.n_stocks,),)
        self.observation_space = spaces.Box(low=-np.inf, 
                                            high=np.inf, 
                                            shape=(self.n_stocks * self.history_usage,))

        self.current_step: int = 0
        self.weights_list: list = []
        self.returns_list: list = []
        


    def reset(self, seed=42):
        """
        doc string

        Good, changing all non-fixed variables inside the environment
        """
        super().reset(seed=None)

        self.current_step = 0
        self.weights = []
        self.portfolio_returns = []

        observation = self.get_observation()
        additional_info = {
            "time_step": self.current_step,
            "cumulative_geo_return": np.cumprod(self.portfolio_returns)
        }

        return observation, additional_info



    def get_observation(self):
        """
        doc string
        """
        start_idx = max(0, self.current_step -self.history_usage)
        end_idx = self.current_step
        observation_space  = self.return_data[start_idx:end_idx].T
        if observation_space.shape[1] < self.history_usage:
            padding = np.zeros((self.n_stocks, self.history_usage - observation_space.shape[1]))
            observation_space = np.hstack([padding, observation_space])
        
        return observation_space.flatten().astype(np.float32)


    def step(self, action):
        """
        doc string
        """
        # Generate weights based on actions
        # Forces action from in range (-1,1) to become (0,1)
        current_weights = (action + 1) / 2                          
        current_weights = (action+1e-8) / (np.sum(action)+1e-8)
        self.weights_list.append(current_weights)
        
        # Find current weights and multiply with weights
        # Variables for (early) stopping
        terminated = self.current_step >= len(self.return_data)-1
        truncated = False
        if not terminated:
            current_returns = self.return_data[self.current_step +1]
            portfolio_return = 0.0
            if self.current_step +1 < len(self.return_data):
                portfolio_return = np.dot(current_weights, current_returns)
            self.returns_list.append(portfolio_return)
        else:
            portfolio_return = 0.0
            self.returns_list.append(portfolio_return)
        # New step
        self.current_step += 1

        # if terminated:
            

        # Returns the next observation space for the algo to use
        next_window = self.get_observation()

        return next_window, portfolio_return, terminated, truncated, {}
        


    def render(self, mode="human"):
        """ 
        doc string
        """
        print(f"Current step: {self.current_step}, and geometric return: {np.cumprod(self.portfolio_returns)}")
        pass

In [33]:
train_env = PortfolioEnvironment(history_usage=history_usage, 
                              return_data=train_data,
                              esg_data=esg_data,
                              objective=objective,
                              esg_compliancy=esg_compliancy)
check_env(train_env, warn=True)

In [47]:
model = SAC(policy="MlpPolicy",
            env=train_env,
            buffer_size=100_000,
            verbose=1
            ).learn(35000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 4.82e+03 |
|    ep_rew_mean     | 93.1     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 30       |
|    time_elapsed    | 641      |
|    total_timesteps | 19268    |
| train/             |          |
|    actor_loss      | -208     |
|    critic_loss     | 5.06     |
|    ent_coef        | 0.0799   |
|    ent_coef_loss   | 19.2     |
|    learning_rate   | 0.0003   |
|    n_updates       | 19167    |
---------------------------------


In [48]:
test_env = PortfolioEnvironment(history_usage=history_usage, 
                              return_data=test_data,
                              esg_data=esg_data,
                              objective=objective,
                              esg_compliancy=esg_compliancy)

In [49]:
obs, info = test_env.reset()
weights_history = []
finished = False

while not finished: 
    action, _ = model.predict(obs, deterministic=True)

    weights = (action+1) / 2
    weights /= np.sum(weights)

    obs, reward, terminated, truncated, info = test_env.step(action)
    finished = terminated or truncated

    weights_history.append(weights)

In [50]:
df = pd.DataFrame(weights_history)

In [51]:
df

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,0.003139,0.072206,0.072762,0.050602,0.060304,0.072598,0.067499,0.001503,0.002399,0.071697,...,0.073187,0.002095,0.073582,0.073236,0.003973,0.008112,0.053282,0.066906,0.002424,0.015928
1,0.003192,0.072122,0.072816,0.051979,0.061784,0.072317,0.067050,0.001612,0.002419,0.071417,...,0.073259,0.002062,0.073477,0.072972,0.004396,0.007348,0.054468,0.065870,0.002606,0.013914
2,0.002901,0.071904,0.072554,0.051282,0.062881,0.072180,0.068906,0.001190,0.001829,0.071746,...,0.073067,0.002080,0.073211,0.072644,0.004026,0.008045,0.054046,0.067195,0.002030,0.015976
3,0.002388,0.072347,0.072679,0.051457,0.057820,0.072513,0.067720,0.001421,0.002009,0.071970,...,0.073018,0.001775,0.072878,0.072834,0.003228,0.008502,0.054018,0.067389,0.003533,0.019396
4,0.002702,0.072331,0.072624,0.051445,0.063167,0.072747,0.066819,0.001493,0.001971,0.072015,...,0.073142,0.002349,0.073470,0.073166,0.003692,0.008262,0.052266,0.067809,0.002281,0.013099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,0.002460,0.073216,0.074271,0.044965,0.072660,0.074396,0.074721,0.000775,0.002139,0.075437,...,0.075224,0.003963,0.075692,0.075828,0.009032,0.015233,0.036243,0.043620,0.000276,0.011624
1201,0.003354,0.074185,0.069472,0.055615,0.072350,0.073512,0.063722,0.001191,0.002776,0.072104,...,0.072417,0.003367,0.074164,0.073929,0.000866,0.010475,0.045513,0.052717,0.002818,0.024081
1202,0.002161,0.073719,0.073560,0.053635,0.061255,0.074400,0.073018,0.000801,0.000938,0.074777,...,0.074522,0.001119,0.075347,0.075373,0.004926,0.007744,0.038826,0.068897,0.003270,0.005132
1203,0.025439,0.067258,0.069417,0.065888,0.063745,0.067862,0.063756,0.004732,0.002459,0.066526,...,0.064962,0.001940,0.072378,0.072519,0.002056,0.007683,0.062650,0.057168,0.007082,0.015120


In [53]:
from stable_baselines3.common.evaluation import evaluate_policy

In [59]:
mean_reward, std_reward = evaluate_policy(model, train_env, n_eval_episodes=100)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

c:\Users\Jakob\OneDrive\Dokumenter\Masteroppgave\MyVenv\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward:-9.96 +/- 0.00


In [ ]:
mean_reward, std_reward = evaluate_policy(model, test_env, n_eval_episodes=100)
print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

c:\Users\Jakob\OneDrive\Dokumenter\Masteroppgave\MyVenv\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward:1.18 +/- 0.00
